# What are Agents?
We can think of agents as enabling “tools” for LLMs. Like how a human would use a calculator for maths or perform a Google search for information — agents allow an LLM to do the same thing.

Using agents allows us to give LLMs access to tools. These tools present an infinite number of possibilities.

---
# Agents and Tools
## To use agents, we require three things:

* A base LLM,
* A tool that we will be interacting with,
* An agent to control the interaction.

---

In [ ]:
!pip install --quiet langchain langchain_openai langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Flow:

```
Initialize Chain  ------>  Create Tool  ----->  Append to Tools  ----> Create Agent ----> Run Agent.
```

1. Build a chain.
2. Create a tool to call that chain.
3. Append all the tool to tools together.
4. Initialize the agent.
5. Run the agent.

# Prebuilt and Custom tool

## Custom Tool


[LLM Math Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html)

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool

In [ ]:
llm = OpenAI(
    temperature=0
)

# Create chain
llm_math_chain = LLMMathChain(llm = llm) # Chain that interprets a prompt and executes python code to do math.


# Create Tool
math_tool = Tool(
    name = 'Calculator',
    description='Useful when some question needs to be answered about math',
    func=llm_math_chain.run # this will call the chain
)


# Append chain to tool
tools = [math_tool]

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [ ]:
tools[0].name, tools[0].description

('Calculator', 'Useful when some question needs to be answered about math')

## Prebuilt tool

Follow this second approach if a prebuilt tool for  use case exists.



In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm = llm
)

In [ ]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

# Agents

We now have the LLM and tools but no agent. To initialize a simple agent, we can do the following:

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent="zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iteration = 3
)

In [ ]:
agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I should use a calculator to solve this problem
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

# Multiple Tools

Adding new tool to already existing tool.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
prompt = PromptTemplate(
    input_variables=['query'],
    template="{query}"
)


# Create Chain
generic_llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

# Create Tool
generic_tool = Tool(
    name = 'Generic QA',
    description='Useful for general purpose question answer',
    func = generic_llm_chain.run

)

# Add to tools
tools.append(generic_tool)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [ ]:
tools[1].name, tools[1].description

('Generic QA', 'Useful for general purpose question answer')

## Inititalize agent

In [ ]:
agent = initialize_agent(
    agent='zero-shot-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iteration=3
)

In [ ]:
agent('What is the capital of India?')



> Entering new AgentExecutor chain...
 I should use a generic QA tool to answer this question.
Action: Generic QA
Action Input: "What is the capital of India?"
Observation: 

The capital of India is New Delhi.
Thought: I now know the final answer.
Final Answer: New Delhi

> Finished chain.


{'input': 'What is the capital of India?', 'output': 'New Delhi'}

In [ ]:
agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I should use a calculator to solve this problem
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [ ]:
agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
 We need to add the number of apples that Mary has with the number of apples that Giorgio brings.
Action: Calculator
Action Input: 4 + (2.5 * 8)
Observation: Answer: 24.0
Thought: We now know the total number of apples.
Action: Generic QA
Action Input: How many apples do we have?
Observation: 

I'm sorry, I cannot answer that question as I do not have access to that information.
Thought: I now know the final answer.
Final Answer: 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': '24 apples.'}